In [ ]:
from pyspark.sql import SparkSession

In [ ]:
# Create a Spark session
spark = SparkSession.builder \
    .appName("KafkaConsumerFraud") \
    .getOrCreate()


In [ ]:
confluentApiKey = ""
confluentSecret = ""
kafkaServer = ""
topicName = "transactions"

In [ ]:
# Read data from the Kafka topic
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafkaServer) \
    .option("kafka.security.protocol", "SASL_SSL") \
    .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(confluentApiKey, confluentSecret)) \
    .option("kafka.sasl.mechanism", "PLAIN") \
    .option("subscribe", topicName) \
    .load()

In [ ]:
# Select the key and value columns from the Kafka data
df = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [ ]:
# Define a query to write the Kafka data to the console
query = df.writeStream \
    .outputMode("append") \
    .format("delta") \
    .option("checkpointLocation", "/transactions/checkpoint/dir") \
    .start("/transactions/table")

# Await termination of the query
query.awaitTermination()

# The awaitTermination() method is used to block the current thread until the streaming query terminates. This is necessary in streaming applications to keep the application running and continuously processing incoming data. Without awaitTermination(), the application would exit immediately after starting the query, and no data would be processed.